In [2]:
%matplotlib inline

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
import twstock as ts
import time 
import threading
import queue
import json
from urllib.request import urlopen

In [59]:
year='2018'

dfa=pd.read_csv(f"C:/Users/黃柏倫/Desktop/data/{year}_fut/{year}_fut.csv",encoding='big5', error_bad_lines=False, index_col=False, dtype='unicode')
dfa=dfa[dfa['契約']=='TX']
dfa.index=range(len(dfa.index))

dfam=[np.nan]
for i in range(1,13):
    u='%02d'%i
    dfam.append(dfa[(dfa['到期月份(週別)']==f'2018{u}  ')])
    dfam[i].index=range(len(dfam[i].index))
for i in range(2,13):
    dfam[i]=dfam[i][dfam[i].交易日期>dfam[i-1].交易日期[len(dfam[i-1].index)-1]]
    dfam[i].index=range(len(dfam[i].index))
#dfam[10]

dpcam=[np.nan]
for i in range(1,13):
    u='%02d'%i
    u1='%02d'%(i+1)
    dpcaw=[np.nan]
    dpca=pd.read_csv(f"C:/Users/黃柏倫/Desktop/data/{year}_opt/{year}_opt_{u}.csv",encoding='big5', error_bad_lines=False, index_col=False, dtype='unicode')
    t=1
    for j in [f'2018{u}W1',f'2018{u}W2',f'2018{u}  ',f'2018{u}W4',f'2018{u}W5',f'2018{u1}W1',f'2018{u1}  ']:
        if list(dpca[(dpca['契約']=='TXO')&(dpca['到期月份(週別)']==j)].index)!=[]:
            dpcaw.append(dpca[(dpca['契約']=='TXO')&(dpca['到期月份(週別)']==j)])
            dpcaw[t].index=range(len(dpcaw[t].index))
            t+=1      
    dpcam.append(dpcaw)
#dpcam[10][2].head()

In [66]:
dfam[12].head()

,交易日期,契約,到期月份(週別),開盤價,最高價,最低價,收盤價,漲跌價,漲跌%,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,價差對單式委託成交量
0,2018/11/22,TX,201812,9766.0,9769.0,9700.0,9714.0,-14,-0.14%,91042,9714,85056,9714,9715,11065,9318,NaN,一般,NaN
1,2018/11/22,TX,201812,9742.0,9788.0,9736.0,9757.0,29,0.30%,32026,-,-,9755,9757,11065,9318,NaN,盤後,NaN
2,2018/11/23,TX,201812,9715.0,9733.0,9613.0,9620.0,-94,-0.97%,113555,9625,86286,9621,9622,11065,9318,NaN,一般,NaN
3,2018/11/23,TX,201812,9720.0,9725.0,9679.0,9714.0,0,0.00%,19950,-,-,9714,9715,11065,9318,NaN,盤後,NaN
4,2018/11/26,TX,201812,9668.0,9850.0,9656.0,9768.0,143,1.49%,147457,9769,87962,9768,9770,11065,9318,NaN,一般,NaN


In [68]:
dpcam[11][-1].head()

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段
0,2018/11/01,TXO,201812,8000.0000,買權,-,-,-,-,0,1800,0,-,-,-,-,NaN,一般
1,2018/11/01,TXO,201812,8000.0000,買權,-,-,-,-,0,-,-,-,-,-,-,NaN,盤後
2,2018/11/01,TXO,201812,8000.0000,賣權,26,27.5,23.5,26,385,26,5287,22.5,26.5,62,22.5,NaN,一般
3,2018/11/01,TXO,201812,8000.0000,賣權,29,29,22.5,25,36,-,-,23,27,62,22.5,NaN,盤後
4,2018/11/01,TXO,201812,8100.0000,買權,-,-,-,-,0,1700,0,-,-,-,-,NaN,一般


In [338]:
def rundetail(m=0,r=100,dif=55,d=len(df.index)):
    profit=0
    h=0
 
    for i in range(d):
 
        s=int(df.loc[i,'開盤價']-df.loc[i,'開盤價']%r)
        s=s-m

        for cd in dpcW:
            m0='%02d'%(month-1)
            if (cd['到期月份(週別)'][0]!=f'{year}{m0}  ')&(df.loc[i,'交易日期']<=cd.loc[len(cd.index)-1,'交易日期']):
                break
        print(f"i={i},{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},{cd['到期月份(週別)'][0]}",end=',')
        
        if i==0:
            s0=s

 
        if (h==1)&(abs(s0-s)>dif):     
            get0m=getp(i,s0,cd,cdm)
            getm=getp(i,s,cd,cdm)  
            profit=profit-get0m+getm                 
            print(f"點位選{s},期貨{int(df.loc[i,'開盤價'])},轉倉 {cd['到期月份(週別)'][0]},{s0}Put和賣{dpcW[-1]['到期月份(週別)'][0]},{s0}Put價格{get0m}-->{cd['到期月份(週別)'][0]},{s}Put和賣{dpcW[-1]['到期月份(週別)'][0]},{s}Put價格{getm}")
            s0=s

        elif h==0:
            getm=getp(i,s,cd,cdm)
            profit=profit+getm
            h=1
            print(f"點位選{s},期貨{int(df.loc[i,'開盤價'])},進場 買{cd['到期月份(週別)'][0]},{s}Put和賣{dpcW[-1]['到期月份(週別)'][0]},{s}Put價格{getm}")
            s0=s
        else:
            
            print(f"點位選{s},期貨{int(df.loc[i,'開盤價'])},沒動作")
 
 
        if (h==1)&(df.loc[i,'交易日期']==cd.loc[len(cd.index)-1,'交易日期'])&(df.loc[i,'交易時段']=='盤後'):
            buym=buyp(i,s0,cd,cdm)
            h=0
            profit=profit+buym  
            print(f"期貨{int(df.loc[i,'收盤價'])},買回 買{cd['到期月份(週別)'][0]},{s0}Put和賣{dpcW[-1]['到期月份(週別)'][0]},{s0}Put價格{buym}")
        
    print(f"手中{h}組,價值{profit}")
    return profit


In [339]:
def run(m=0,r=100,dif=55,d=len(df.index)):
    profit=0
    h=0
 
    for i in range(d):
 
        s=int(df.loc[i,'開盤價']-df.loc[i,'開盤價']%r)
        s=s-m

        for cd in dpcW:
            m0='%02d'%(month-1)
            if (cd['到期月份(週別)'][0]!=f'{year}{m0}  ')&(df.loc[i,'交易日期']<=cd.loc[len(cd.index)-1,'交易日期']):
                break
        #print(f"i={i},{df.loc[i,'交易日期']}{df.loc[i,'交易時段']},{cd['到期月份(週別)'][0]}",end=',')
        
        if i==0:
            s0=s

 
        if (h==1)&(abs(s0-s)>dif):     
            get0m=getp(i,s0,cd,cdm)
            getm=getp(i,s,cd,cdm)  
            profit=profit-get0m+getm                 
            #print(f"點位選{s},期貨{int(df.loc[i,'開盤價'])},轉倉 {cd['到期月份(週別)'][0]},{s0}Put和賣{dpcW[-1]['到期月份(週別)'][0]},{s0}Put價格{get0m}-->{cd['到期月份(週別)'][0]},{s}Put和賣{dpcW[-1]['到期月份(週別)'][0]},{s}Put價格{getm}")
            s0=s

        elif h==0:
            getm=getp(i,s,cd,cdm)
            profit=profit+getm
            h=1
            #print(f"點位選{s},期貨{int(df.loc[i,'開盤價'])},進場 買{cd['到期月份(週別)'][0]},{s}Put和賣{dpcW[-1]['到期月份(週別)'][0]},{s}Put價格{getm}")
            s0=s
#        else:
            
            #print(f"點位選{s},期貨{int(df.loc[i,'開盤價'])},沒動作")
 
 
        if (h==1)&(df.loc[i,'交易日期']==cd.loc[len(cd.index)-1,'交易日期'])&(df.loc[i,'交易時段']=='盤後'):
            buym=buyp(i,s0,cd,cdm)
            h=0
            profit=profit+buym  
            #print(f"期貨{int(df.loc[i,'收盤價'])},買回 買{cd['到期月份(週別)'][0]},{s0}Put和賣{dpcW[-1]['到期月份(週別)'][0]},{s0}Put價格{buym}")
        
    #print(f"手中{h}組,價值{profit}")
    return profit


In [340]:
def getp(i,s,cd,cdm):
    wp=float(cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['開盤價'].iloc[1])
    mp=float(cdm[(cdm['履約價']==s)&(cdm['交易日期']==df.loc[i,'交易日期'])&(cdm['交易時段']==df.loc[i,'交易時段'])]['開盤價'].iloc[1])
    return +wp-mp

def buyp(i,s,cd,cdm):

    wp=float(cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]['收盤價'].iloc[1])
    mp=float(cdm[(cdm['履約價']==s)&(cdm['交易日期']==df.loc[i,'交易日期'])&(cdm['交易時段']==df.loc[i,'交易時段'])]['收盤價'].iloc[1])
    return -wp+mp



In [372]:
month=12
df=dfam[month]

df.開盤價=df.開盤價.astype(float)
df.最高價=df.最高價.astype(float)
df.最低價=df.最低價.astype(float)
df.收盤價=df.收盤價.astype(float)
dpcW=[]
for i in range(6):
    if i<3:
        dpcW.append(dpcam[month-1][i-4])
    elif i<5:
        dpcW.append(dpcam[month][i-2])
    else:
        dpcW.append(pd.concat([dpcam[month-1][-1],dpcam[month][i-2]]))
        dpcW[i].index=range(len(dpcW[i].index))
    dpcW[i]=dpcW[i].replace('-',0)
    dpcW[i].履約價=dpcW[i].履約價.astype(float)
cdm=dpcW[-1]

In [373]:
m=0
r=100
dif=1000
d=len(df.index)
print(f"{month}月")
for dif in np.arange(100,1005,50):
    print(f"價差{dif}轉倉,獲益{run(m,r,dif,d)}")

12月
價差100轉倉,獲益61.10000000000001
價差150轉倉,獲益61.10000000000001
價差200轉倉,獲益62.1
價差250轉倉,獲益62.1
價差300轉倉,獲益95.0
價差350轉倉,獲益95.0
價差400轉倉,獲益95.0
價差450轉倉,獲益95.0
價差500轉倉,獲益95.0
價差550轉倉,獲益95.0
價差600轉倉,獲益95.0
價差650轉倉,獲益95.0
價差700轉倉,獲益95.0
價差750轉倉,獲益95.0
價差800轉倉,獲益95.0
價差850轉倉,獲益95.0
價差900轉倉,獲益95.0
價差950轉倉,獲益95.0
價差1000轉倉,獲益95.0


3價差1000轉倉,獲益-48.199999999999996
價差1000轉倉,獲益1.3999999999999986
價差1000轉倉,獲益-4.0
價差1000轉倉,獲益44.39999999999999
價差1000轉倉,獲益44.400000000000006
價差1000轉倉,獲益-3.799999999999997
價差1000轉倉,獲益-34.5
價差1000轉倉,獲益21.5
價差1000轉倉,獲益120.5
12價差1000轉倉,獲益95.0

In [374]:

rundetail(m,r,dif,d)

i=0,2018/11/22一般,201811W4,點位選9700,期貨9766,進場 買201811W4,9700Put和賣201812  ,9700Put價格-105.0
i=1,2018/11/22盤後,201811W4,點位選9700,期貨9742,沒動作
i=2,2018/11/23一般,201811W4,點位選9700,期貨9715,沒動作
i=3,2018/11/23盤後,201811W4,點位選9700,期貨9720,沒動作
i=4,2018/11/26一般,201811W4,點位選9600,期貨9668,沒動作
i=5,2018/11/26盤後,201811W4,點位選9600,期貨9620,沒動作
i=6,2018/11/27一般,201811W4,點位選9700,期貨9727,沒動作
i=7,2018/11/27盤後,201811W4,點位選9700,期貨9768,沒動作
i=8,2018/11/28一般,201811W4,點位選9700,期貨9780,沒動作
i=9,2018/11/28盤後,201811W4,點位選9700,期貨9729,沒動作
期貨9764,買回 買201811W4,9700Put和賣201812  ,9700Put價格131.0
i=10,2018/11/29一般,201812W1,點位選9900,期貨9984,進場 買201812W1,9900Put和賣201812  ,9900Put價格-66.0
i=11,2018/11/29盤後,201812W1,點位選9800,期貨9879,沒動作
i=12,2018/11/30一般,201812W1,點位選9900,期貨9900,沒動作
i=13,2018/11/30盤後,201812W1,點位選9800,期貨9872,沒動作
期貨9893,買回 買201812W1,9900Put和賣201812  ,9900Put價格81.0
i=14,2018/12/03一般,201812W1,點位選10000,期貨10012,進場 買201812W1,10000Put和賣201812  ,10000Put價格-86.0
i=15,2018/12/03盤後,201812W1,點位選9800,期貨9855,沒動作
i=16,2018/12/04一般,201812W1,點位選10000,期貨

95.0

In [341]:
#i=19,2018/10/31盤後,201810W5,點位選9500,期貨9510,沒動作
#期貨9624,買回 買201810W5,9400Put和賣201811  ,9400Put價格117.0
i=10
cd=dpcW[1]
s=9400
cd[(cd['履約價']==s)&(cd['交易日期']==df.loc[i,'交易日期'])&(cd['交易時段']==df.loc[i,'交易時段'])]

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段


In [342]:
cdm[(cdm['履約價']==s)&(cdm['交易日期']==df.loc[i,'交易日期'])&(cdm['交易時段']==df.loc[i,'交易時段'])]

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段
3568,2018/05/24,TXO,201806,9400.0,買權,0,0,0,0,0,1520,21,0,0,0,0,NaN,一般
3570,2018/05/24,TXO,201806,9400.0,賣權,4.5,5.1,4.4,4.4,1211,4.4,8976,4.4,4.6,1140,1.3,NaN,一般
